In [ ]:
import xuleta as xu
import numpy as np
import pylab as plt
import bottleneck as bn
from glob import glob
import os

### MONTHLY MEANS
Because no-data values screw up cummulative sums

In [ ]:
folder = "/media/denis/seagate/ESI_2017/brasil_04gf/"
tipo = 'Float32'
dictipo = {'Float32':3, 'Int16':1}

In [ ]:
if not os.path.exists(folder+"monthly_mean"):
    os.makedirs(folder+"monthly_mean")
    
folderout = folder+"monthly_mean/"

In [ ]:
l = sorted(glob(folder+"*.tif"))
l2 = []
for i in l:
    l2.append(i.split("/")[-1][:-7])
l2 = [ii for n,ii in enumerate(l2) if ii not in l2[:n]]

In [ ]:
for i in l2:
    toOpen = []
    for j in l:
        if i == str(j.split("/")[-1][0:7]):
            toOpen.append(j)
    print(toOpen)
    
    arrays = []
    for k in toOpen:
        imagem, meta = xu.TifToArray(k)
        imagem = np.where(imagem == -9999, np.nan, imagem)
        arrays.append(imagem)
        arrays2 = np.dstack(arrays)
    xu.ArrayToTif(np.nanmean(arrays2, axis=2), i+'.tif', folderout, meta, Type=dictipo[tipo])  
    arrays, media, toOpen, arrays2 = None, None, None, None  

In [ ]:
os.chdir(folderout)
os.system("for f in *.tif; do gdalwarp -overwrite  -ot Float32 -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=9 -multi -dstnodata -9999 -of GTiff $f comp$f; done")
xu.findreplace(folder=folderout,find="comp",replace="")

In [ ]:

xu.findreplace("/media/denis/seagate/PB/MODIS/mcd43a3/monthly_mean/",find="mean",replace="")

### PERIOD MEAN
To calculate the mean for wet, dry and whatever season

In [ ]:
folder = "/media/denis/seagate/PB/MODIS/myd17a2h/gpp/monthly_mean/"
tipo = 'Int16'
dictipo = {'Float32':3, 'Int16':1}

In [ ]:
#SOMA ANUAL
for y in years:
    
    toOpen = []
    for i in l:
        #print(i.split("/")[-1][0:4]+y)
        if y == str(i.split("/")[-1][0:4]):
            toOpen.append(i)
            print(i.split("/")[-1]+"__"+y)
    
    arrays = []
    for j in toOpen:
        imagem, meta = xu.TifToArray(j)
        
        arrays.append(imagem)
        arrays2 = np.dstack(arrays)
    xu.ArrayToTif(np.nanmean(arrays, axis=2), 'sum'+y+'.tif', folderout, meta, Type=1)    

In [ ]:
os.chdir(folderout)
os.system('for f in ym*.tif; do gdalwarp -ot Int16  -dstnodata 0 -r near -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -wo OPTIMIZE_SIZE=TRUE $f year$f; done')
os.system('rm ym*.tif')

### YEARLY data

In [ ]:
l = sorted(glob(folderout+"*.tif"))
l[0].split("/")[-1][5:9]

In [ ]:
#media ANUAL (agora sim, sem os nodata)
for y in years:
    
    toOpen = []
    for i in l:
        #print(i.split("/")[-1][0:4]+y)
        if y == str(i.split("/")[-1][5:9]):
            toOpen.append(i)
            print(i.split("/")[-1]+"__"+y)
    
    arrays = []
    for j in toOpen:
        imagem, meta = xu.TifToArray(j)
        
        arrays.append(imagem)
        arrays2 = np.dstack(arrays)
    
    xu.ArrayToTif(np.nanmean(arrays2, axis=2), 'ymean'+y+'.tif', folderout, meta, Type=1) 